In [1]:
import sys
import random
import PySide6
from PySide6 import QtCore, QtWidgets, QtGui
from PySide6.QtWidgets import *
from PySide6.QtCore import Qt
from PySide6.QtGui import QPalette, QColor
import numpy as np
import pandas as pd


In [2]:
data=None

try:
    data=pd.read_feather("data/stage1_partial.feather")
except:
    try:
        print("Loading backup, main data corrupted.")
        data=pd.read_feather("data/stage1_partial.backup.feather")
    except:
        print("Loading stage 0, main data & backup corrupted.")
        data=pd.read_feather("data/stage0.feather")


In [3]:
(1-data[data["volcano_certainty"]==-1].shape[0]/data.shape[0])*100

1.0697674418604697

In [107]:
data[data["volcano_certainty"]==-1].head()

,path,volcano_certainty,day_night,has_fume,is_explosion,predicted,certainty
9729,images\2015\sep\p0912155.jpg,-1,-1,-1,-1,FUM,0.998997
9730,images\2005\dic\p1222053.jpg,-1,-1,-1,-1,FUM,0.998996
9731,images\2011\dic\p1214111.jpg,-1,-1,-1,-1,FUM,0.998995
9732,images\2012\oct\p1021126.jpg,-1,-1,-1,-1,FUM,0.998991
9733,images\2014\oct\p1015141.jpg,-1,-1,-1,-1,FUM,0.998987


In [108]:
def save(is_backup=False):
    data.to_feather(f"data/stage1_partial{'.backup' if is_backup else ''}.feather")
save(True)

In [109]:
data[data["volcano_certainty"]==-1].index

Index([ 9729,  9730,  9731,  9732,  9733,  9734,  9735,  9736,  9737,  9738,
       ...
       19340, 19341, 19342, 19343, 19344, 19345, 19346, 19347, 19348, 19349],
      dtype='int64', length=4044)

In [110]:
def get_next():
    idx=data[data["volcano_certainty"]==-1].index
    if(idx.shape[0]==0):
        return None, 0, None
    return idx[0],idx.shape[0], data.iloc[idx[0]]

In [111]:
next_idx,remainder,row=get_next()

In [112]:
def safe_update(idx, values):
    save(True)
    data.iloc[idx,1:len(values)+1]=values
    save(False) 

In [113]:
keys = ["volcano_certainty", "day_night", "has_fume", "is_explosion"]
labels = {
    keys[0]:"No volcán",
    keys[1]:"Noche",
    keys[2]:"Sin fumarola",
    keys[3]:"No explotando",
}
endLabels = {
    keys[0]:"Es volcán",
    keys[1]:"Día",
    keys[2]:"Tiene fumarola",
    keys[3]:"Está explotando",
}

In [114]:
window=None

In [115]:
class ImageDisplay(QWidget):

    def loadpixmap(self, path):
        self.pixmap=QtGui.QPixmap(path)#.scaledToHeight(self.geometry().width())
        self.label.setPixmap(self.pixmap)
        
    def __init__(self, path = ""):
        super().__init__()
        label=QLabel()
        label.setAlignment(Qt.Alignment.AlignCenter)
        label.setScaledContents(True)
        self.label=label
        self.loadpixmap(path)
        
        self.layout=QGridLayout()
        self.layout.addWidget(label, 0,0)
        self.setLayout(self.layout)

In [116]:
class DataLayout(QWidget):

    @QtCore.Slot()
    def value_changed(self):
        self.values=[]
        for slider in self.sliders:
            self.values.append(slider.value())
        
    @QtCore.Slot()
    def button_clicked(self):
        global next_idx,remainder,row
        safe_update(next_idx, self.values)
        next_idx,remainder,row=get_next()
        window.image.loadpixmap(row["path"])
        #for slider in self.sliders:
        #    slider.setValue(0)
        self.values=[]
        for slider in self.sliders:
            self.values.append(slider.value())
        self.progress.setText(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%) Suggested: {row['predicted']} {row['certainty']*100:.02F}%")
        #window.close()
        
    
    def __init__(self):
        super().__init__()
        self.setMinimumWidth(600)
        self.setMaximumWidth(600)
        layout=QGridLayout()

        self.sliders=[]
        self.values=[0 for k in keys]
        self.progress = QLabel(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%) Suggested: {row['predicted']} {row['certainty']*100:.02F}%")
        self.progress.setMaximumHeight(50)
        self.progress.setAlignment(Qt.Alignment.AlignCenter)
        layout.addWidget(self.progress,0,0,1,6)
        
        for i,key in enumerate(keys):
            slider=QSlider(Qt.Orientation.Horizontal)
            slider.setMinimumHeight(50)
            if(key=="volcano_certainty" or key=="day_night"):
                slider.setSingleStep(10)
                slider.setPageStep(10)
            else:
                slider.setSingleStep(4)
                slider.setPageStep(4)
            slider.setMinimum(0)
            slider.setMaximum(10)
            slider.setTickPosition(QSlider.TicksBelow)
            slider.setSizePolicy(QSizePolicy.MinimumExpanding,QSizePolicy.Fixed)
            slider.valueChanged.connect(self.value_changed)
            self.sliders.append(slider)
            
            label=QLabel(labels[key])
            label.setMinimumHeight(50)
            label.setAlignment(Qt.Alignment.AlignCenter)
            
            label2=QLabel(endLabels[key])
            label2.setMinimumHeight(50)
            label2.setAlignment(Qt.Alignment.AlignCenter)
            
            layout.addWidget(label, 1+i,0)
            layout.addWidget(slider, 1+i,1,1,4)
            layout.addWidget(label2, 1+i,5)

        button=QPushButton("Siguiente")
        button.setMinimumHeight(50)
        #button.setAlignment(Qt.Alignment.AlignCenter)
        button.clicked.connect(self.button_clicked)
        #layout.addWidget(button,len(keys)+1,1, 1,4)
        
        #layout.addRow(label1,slider1)
        #slider2=QSlider(Qt.Orientation.Horizontal)
        #layout.addRow("TEST",slider2)
        self.setLayout(layout)

In [117]:

class KeyPressFilter(QtCore.QObject):

    def eventFilter(self, widget, event):
        if event.type() == QtCore.QEvent.KeyPress:
            key = event.keyCombination().key()
            if key==Qt.Key.Key_Return:
                window.data.button_clicked()
        return False

In [118]:
class MainWindow(QMainWindow):

    def __init__(self):
        super(MainWindow, self).__init__()

        self.setWindowTitle("Classifier")

        layout = QHBoxLayout()
        left = ImageDisplay(row["path"])
        right= DataLayout()

        self.data=right
        self.image=left

        layout.addWidget(left)
        layout.addWidget(right)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        self.eventFilter = KeyPressFilter(parent=self)
        self.installEventFilter(self.eventFilter)

In [129]:
if __name__ == "__main__" and remainder>0:
    if not QtWidgets.QApplication.instance():
        app = QtWidgets.QApplication([])
    else:
        app = QtWidgets.QApplication.instance()
    font=QtGui.QFont("Segoe UI", 14)
    app.setFont(font)
    app.setStyle(QStyleFactory.create("windows"))
    window = MainWindow()
    window.resize(1900,1000)
    window.showMaximized()

    app.exec()

TypeError: 'NoneType' object is not subscriptable

In [127]:
cond=(data["predicted"]=="EXP+FUM") & (data["certainty"]>=.92) & (data["volcano_certainty"]==-1)
data[cond]

,path,volcano_certainty,day_night,has_fume,is_explosion,predicted,certainty
17392,images\2018\mar\p0306183.jpg,-1,-1,-1,-1,EXP+FUM,0.998993
17393,images\2019\ago\p0817198.jpg,-1,-1,-1,-1,EXP+FUM,0.998988
17394,images\2017\jul\p0709171.jpg,-1,-1,-1,-1,EXP+FUM,0.998983
17395,images\2014\sep\p0909141.jpg,-1,-1,-1,-1,EXP+FUM,0.998983
17396,images\2014\ago\p0817146.jpg,-1,-1,-1,-1,EXP+FUM,0.998981
...,...,...,...,...,...,...,...
18723,images\2016\jul\p0728164.jpg,-1,-1,-1,-1,EXP+FUM,0.920771
18724,images\2012\ene\p0113122.jpg,-1,-1,-1,-1,EXP+FUM,0.920658
18725,images\2018\jul\p0716182.jpg,-1,-1,-1,-1,EXP+FUM,0.920171
18726,images\2014\feb\p0207144.jpg,-1,-1,-1,-1,EXP+FUM,0.920038


In [128]:
data.loc[cond, "volcano_certainty"]=10
data.loc[cond, "day_night"]=2
data.loc[cond, "has_fume"]=8
data.loc[cond, "is_explosion"]=8
data[cond]

,path,volcano_certainty,day_night,has_fume,is_explosion,predicted,certainty
17392,images\2018\mar\p0306183.jpg,10,2,8,8,EXP+FUM,0.998993
17393,images\2019\ago\p0817198.jpg,10,2,8,8,EXP+FUM,0.998988
17394,images\2017\jul\p0709171.jpg,10,2,8,8,EXP+FUM,0.998983
17395,images\2014\sep\p0909141.jpg,10,2,8,8,EXP+FUM,0.998983
17396,images\2014\ago\p0817146.jpg,10,2,8,8,EXP+FUM,0.998981
...,...,...,...,...,...,...,...
18723,images\2016\jul\p0728164.jpg,10,2,8,8,EXP+FUM,0.920771
18724,images\2012\ene\p0113122.jpg,10,2,8,8,EXP+FUM,0.920658
18725,images\2018\jul\p0716182.jpg,10,2,8,8,EXP+FUM,0.920171
18726,images\2014\feb\p0207144.jpg,10,2,8,8,EXP+FUM,0.920038


In [104]:
data.groupby(by="predicted")["certainty"].describe()

,count,mean,std,min,25%,50%,75%,max
predicted,,,,,,,,
EXP,248.0,0.901689,0.150637,0.413489,0.863572,0.990549,0.999558,0.999999
EXP+FUM,7257.0,0.984810,0.062625,0.364781,0.999202,0.999959,0.999996,1.000000
FUM,8501.0,0.984022,0.063728,0.374945,0.999016,0.999953,0.999997,1.000000
INA,2756.0,0.981116,0.065534,0.444901,0.998295,0.999954,0.999998,1.000000
UNK,588.0,0.839406,0.297666,0.000000,0.833412,0.987401,0.996927,0.998998


In [130]:
data

,path,volcano_certainty,day_night,has_fume,is_explosion,predicted,certainty
0,images\2006\jun\p0622063.jpg,10,5,4,0,UNK,0.998998
1,images\2023\jul\p0704235.jpg,0,5,4,0,UNK,0.998997
2,images\2013\sep\p0926132.jpg,0,5,4,0,UNK,0.998995
3,images\2019\ene\p0120193.jpg,10,0,0,0,UNK,0.998994
4,images\2018\jul\p0723181.jpg,10,10,4,0,UNK,0.998969
...,...,...,...,...,...,...,...
19345,images\2015\may\p0530155.jpg,0,10,4,4,EXP,0.489222
19346,images\2016\oct\p1028164.jpg,0,10,4,4,EXP,0.461512
19347,images\2018\dic\p1231184.jpg,0,10,4,4,EXP,0.459877
19348,images\2018\jun\p0603184.jpg,0,10,4,4,EXP,0.421862


In [131]:
save(True)

In [132]:
save(False)